## Step1 ##

In [1]:
# Iniziamo con l'importazione delle librerie che ci saranno utili nello svolgimento delle task:
import os
import shutil
import csv

# Definiamo i percorsi di origine e le sottocartelle di destinazione
# Impostiamo in una variabile (stringa) il nome della cartella di origine:
source_folder = 'files'

# Creiamo un dict in cui ogni key (tipologia di file) corrisponde a un nuovo percorso di destinazione
# Il metodo path.join() permette infatti di assemblare più componenti di path restituendone uno nuovo:
dest_folders = {
    'audio': os.path.join(source_folder, 'audio'),
    'doc': os.path.join(source_folder, 'docs'),
    'image': os.path.join(source_folder, 'images')
}

# Definiamo con lo stesso metodo dove andrà il file di riepilogo ('recap.csv'):
recap_file = os.path.join(source_folder, 'recap.csv')

# Creiamo le sottocartelle se non esistono:
# Serviamoci di un ciclo per accedere a tutti i valori (cartelle) del dict "dest_folders"
for folder in dest_folders.values():
    """
    Verifica e crea le sottocartelle di destinazione per ogni tipo di file.
    La funzione 'os.makedirs()' viene usata per creare la cartella se non esiste già.
    Il parametro 'exist_ok=True' assicura che non venga sollevato un errore se la cartella è già presente.
    """
    os.makedirs(folder, exist_ok=True)

# Per posizionare i file nella cartella "giusta" è opportuno specificare un fattore discriminante, che in questo caso è l'estensione del file.
# Questa operazione è fattibile usando anche qui un dict, dove come chiavi abbiamo i nomi delle cartelle di destinazione e come valori le rispettive estensioni valide:
file_types = {
    'audio': ['.mp3', '.wav', '.flac'],
    'doc': ['.txt', '.doc', '.docx', '.pdf', '.odt'],
    'image': ['.jpg', '.jpeg', '.png', '.gif']
}

# Passiamo ora al file di riepilogo e assicuriamoci che non esista già grazie al metodo os.path.exists():
if not os.path.exists(recap_file):
    """
    Verifica se il file di riepilogo ('recap.csv') esiste. 
    Se non esiste, lo crea e scrive la riga di intestazione con i nomi delle colonne.
    """
    with open(recap_file, mode='w', newline='', encoding='utf-8') as file:
        # Questo metodo converte il file dato in CSV:
        writer = csv.writer(file)
        # Scriviamo i nomi delle colonne del file di riepilogo:
        writer.writerow(['Name', 'Type', 'Size (bytes)'])

# Iteriamo sui file nella cartella di origine in ordine alfabetico:
for filename in sorted(os.listdir(source_folder)):
    """
    Ciclo per ogni file nella cartella di origine. Il file viene ordinato alfabeticamente
    prima di essere analizzato per determinarne il tipo e spostarlo nella giusta cartella.
    """
    # Definiamo in una variabile il path "tipo" che avrà ogni file:
    file_path = os.path.join(source_folder, filename)
    
    # Verifichiamo se l'elemento è un file (non una cartella):
    if os.path.isfile(file_path):
        """
        Verifica se l'elemento corrente è un file e non una directory.
        La funzione 'os.path.isfile()' restituisce True se l'elemento è un file, altrimenti False.
        """
        # Otteniamo la dimensione del file:
        file_size = os.path.getsize(file_path)
        # Otteniamo l'estensione del file e impostiamola in minuscolo per omogeneità:
        file_ext = os.path.splitext(filename)[1].lower()
        # Inizializziamo la variabile che descrive il tipo di file:
        file_type = None
        
        # Determiniamo il tipo di file in base all'estensione:
        for type_name, extensions in file_types.items():
            """
            Itera attraverso ogni tipo di file (audio, doc, image) e le rispettive estensioni.
            Se l'estensione del file corrente è presente nel dict, assegna il tipo di file alla variabile.
            """
            if file_ext in extensions:
                file_type = type_name
                # Interrompiamo il ciclo se viene trovato un tipo di file corrispondente
                break
        
        # Se è un file di tipo riconosciuto, stampiamo le informazioni e lo spostiamo nella cartella di appartenenza:
        if file_type:
            """
            Se il file è di un tipo riconosciuto, vengono stampate le informazioni (nome, tipo e dimensione),
            il file viene spostato nella relativa sottocartella e le informazioni vengono registrate nel file recap.csv.
            """
            print(f"{filename} type:{file_type} size:{file_size}B")
            
            # Spostiamo il file nella relativa sottocartella:
            shutil.move(file_path, dest_folders[file_type])
            
            # Scriviamo le informazioni nel file recap.csv:
            with open(recap_file, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow([filename, file_type, file_size])


vestrahorn-mountain-iceland-80oe_l.jpeg type:image size:431912B


## Step3 ##

In [ ]:
# Importazione delle librerie di interesse:
import os
from PIL import Image
import numpy as np
from tabulate import tabulate

# Percorso relativo alla cartella images
images_folder = os.path.join("files", "images")

# Verifica che la cartella esista
if not os.path.exists(images_folder):
    raise FileNotFoundError(f"La cartella '{images_folder}' non esiste. Verifica il percorso.")

# Funzione per analizzare un'immagine
def analyze_image(image_path):
    # trasformiamo l'immagine in variabile
    img = Image.open(image_path)
    # traduciamo l'immagine creata sottoforma di array
    img_array = np.array(img)
    
    # Impostiamo sottoforma di variabili le dimensioni dell'immagine (metodi)
    height, width = img.height, img.width
    
    # Determinazione del numero di canali
    # Impostiamo la condizione in cui l'immagine sia una scala di grigi (2 dimensioni)
    if len(img_array.shape) == 2:  # Scala di grigi
        # Calcoliamo la media:
        grayscale = np.mean(img_array)
        # Settiamo gli altri colori e la trasparenza a zero
        r = g = b = alpha = 0.0
    # Gestiamo il caso in cui l'immagine sia RGB o RGBA (3-4 dimensioni)
    elif len(img_array.shape) == 3:
        # caso in cui la terza dimensione sia costituita da tre matrici (RGB)
        if img_array.shape[2] == 3:  # RGB
            # settiamo la variabile grayscale a 0
            grayscale = 0.0
            # in tutte e tre le matrici iteriamo la media
            r, g, b = [np.mean(img_array[..., i]) for i in range(3)]
            # settiamo la variabile trasparenza a zero
            alpha = 0.0
        # caso in cui si tratti di un immagine RGBA, ovvero possiede 4 matrici
        elif img_array.shape[2] == 4:  # RGBA
            # settiamo la variabile grayscale a 0:
            grayscale = 0.0
            # calcoliamo la media iterata sulle 4 matrici
            r, g, b, alpha = [np.mean(img_array[..., i]) for i in range(4)]
    # Gestiamo il caso in cui non sia riconosciuto il formato dell'immagine
    else:
        raise ValueError(f"Formato immagine non riconosciuto: {image_path}")
    # la funzione alla fine restituirà le variabili che andranno a comporre la tabella riassuntiva    
    return os.path.basename(image_path), height, width, grayscale, r, g, b, alpha

# Iterazione sulle immagini nella cartella
# impostiamo una lista che conterrà i risultati della funzione
results = []
# iteriamo ogni immagine della cartella 'images':
for filename in os.listdir(images_folder):
    # verifichiamo l'estensione dei file (se si trattano di immagini)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        # trasformiamo ogni immagine in variabile da inserire nella funzione
        image_path = os.path.join(images_folder, filename)
        # applichiamo la funzione e inseriamo i risultati nella lista
        results.append(analyze_image(image_path))

# Creazione della tabella riassuntiva
# inseriamo in una lista quelli che saranno i titoli delle colonne:
headers = ["name", "height", "width", "grayscale", "R", "G", "B", "ALPHA"]
# con la funzione tabulate creiamo la tabella specificando i risultati e i nomi delle colonne da prendere dalle rispettive liste:
table = tabulate(results, headers=headers, tablefmt="fancy_grid")

# Stampa della tabella
print(table)


╒═════════════════════════════════════════╤══════════╤═════════╤═════════════╤══════════╤══════════╤═══════════╤═════════╕
│ name                                    │   height │   width │   grayscale │        R │        G │         B │   ALPHA │
╞═════════════════════════════════════════╪══════════╪═════════╪═════════════╪══════════╪══════════╪═══════════╪═════════╡
│ bw.png                                  │      512 │     512 │     21.4801 │   0      │   0      │   0       │   0     │
├─────────────────────────────────────────┼──────────┼─────────┼─────────────┼──────────┼──────────┼───────────┼─────────┤
│ daffodil.jpg                            │      500 │     335 │      0      │ 109.227  │  85.5231 │   4.76619 │   0     │
├─────────────────────────────────────────┼──────────┼─────────┼─────────────┼──────────┼──────────┼───────────┼─────────┤
│ eclipse.png                             │      256 │     256 │      0      │ 109.05   │ 109.522  │  39.849   │ 133.591 │
├───────────────